# Set up environment

In [1]:
# !pip install --upgrade folium
# !pip install --upgrade numpy
# !pip install --upgrade pandas
# !pip install --upgrade matplotlib
# !pip install --upgrade seaborn
# !pip install colour

In [2]:
import datetime

from collections import defaultdict

import os

from typing import List

import re

In [3]:
import numpy as np

import pandas as pd

In [4]:
from colour import Color

import folium 

import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable

import seaborn as sns
sns.set()

In [5]:
import ipywidgets as widgets
from ipywidgets import interact

# Preloading

In [6]:
datasets_dir_path = 'data'

## Preloading: preload CSV

In [7]:
csv_datasets_dir_path = os.path.join(
    datasets_dir_path,
    'csv'
)

#### The main air pollution dataset

#### dataset src: http://201.245.192.252:81/home/map

In [8]:
air_data_file_name = 'RMCAB_air_quality_sensor_data.csv'
air_data_file_path = os.path.join(
    csv_datasets_dir_path,
    air_data_file_name
)

air_data_df = pd.read_csv(
    air_data_file_path
)

#### Stations locations dataset

In [9]:
location_data_file_name = 'stations_loc.csv'
location_data_file_path = os.path.join(
    csv_datasets_dir_path,
    location_data_file_name
)

location_data_df = pd.read_csv(
    location_data_file_path
)

# Overview

In [10]:
def overview_df(
    df: pd.DataFrame
) -> None:
    display(
        df.sample(10),
        df.shape,
        df.isnull().sum(),
        df.dtypes
    )

In [11]:
overview_df(air_data_df)

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime
72476,16.8,15.0,1.180,8.449,9.628,0.26756,8.945,CDAR,10-04-2021 21:00
62686,35.5,22.5,23.474,26.523,49.997,1.11024,NaN,CBV,26-02-2021 23:00
41051,9.3,3.0,6.911,7.697,14.606,NaN,20.385,LFR,08-09-2021 12:00
116445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CSE,17-04-2021 22:00
119642,55.0,23.0,NaN,NaN,NaN,0.87674,3.398,CSE,29-08-2021 03:00
104548,71.8,44.2,26.813,16.124,42.937,0.60685,1.558,JAZ,08-12-2021 05:00
159099,17.3,7.0,0.638,13.553,14.191,0.52964,26.818,KEN,01-03-2021 04:00
146248,45.2,11.8,17.079,15.788,32.867,NaN,NaN,MOV2,11-09-2021 17:00
116543,86.0,51.0,NaN,NaN,NaN,0.57955,3.003,CSE,21-04-2021 24:00
143130,29.0,11.3,NaN,NaN,NaN,NaN,NaN,MOV2,04-05-2021 19:00


(166440, 9)

PM10        20014
PM2.5       15312
NO          27664
NO2         27662
NOX         27668
CO          31238
OZONO       32132
Station         0
DateTime        0
dtype: int64

PM10        float64
PM2.5       float64
NO          float64
NO2         float64
NOX         float64
CO          float64
OZONO       float64
Station      object
DateTime     object
dtype: object

In [12]:
air_data_df['DateTime'].duplicated().sum()

157680

In [13]:
air_data_df['Station'].value_counts(dropna=False, normalize=True)

Station
USM     0.052632
FTB     0.052632
USQ     0.052632
MOV2    0.052632
COL     0.052632
GYR     0.052632
CSE     0.052632
7MA     0.052632
JAZ     0.052632
SCR     0.052632
BOL     0.052632
CDAR    0.052632
CBV     0.052632
MAM     0.052632
PTE     0.052632
LFR     0.052632
TUN     0.052632
SUB     0.052632
KEN     0.052632
Name: proportion, dtype: float64

In [14]:
overview_df(location_data_df)

,estacion,Sigla,Latitud,Longitud,Altitud (m),Altura (m),Localidad,Tipo de zona,Tipo de estación,Dirección
13,san_cristobal,SCR,"4°34'21.19""N","74°5'1.73""W",2688,0,San Cristóbal,Urbana,De fondo,Carrera 2 Este # 12-78 sur
17,ciudad_bolivar,CBV,"4°34'40.1""N","74°09'58.6""W",2661,0,Ciudad Bolívar,Urbana,Residencial,Calle 70 Sur # 56 - 11
12,tunal,TUN,"4°34'34.41""N","74°7'51.44""W",2589,0,Tunjuelito,Urbana,De fondo,Carrera 24 # 49-86 sur
11,carvajal_sevillana,CSE,"4°35'45.0""N","74°08'54.6""W",2563,3,Kennedy,Urbana,Tráfico/industrial,Autopista Sur # 63-40
16,bosa,BOS,"4°36'20.2""N","74°12'14.6""W",2546,0,Bosa,Urbana,De fondo,Diagonal 73 F Sur # 78 - 44
14,el_jazmin,JAZ,"4°36'30.6""N","74°06'53.8""W",2559,0,Puente Aranda,Urbana,Residencial,Calle 1 G # 41 A 39
4,las_ferias,LFR,"4°41'26.52""N","74°4'56.94""W",2552,0,Engativá,Urbana,De tráfico,Avenida Calle 80 # 69Q-50
0,guaymaral,GYR,"4°47'01.5""N","74°02'38.9""W",2580,0,Suba,Sub urbana,De fondo,Autopista Norte # 205-59
1,usaquen,USQ,"4°42'37.26""N","74°1'49.50""W",2570,10,Usaquén,Urbana,De fondo,Carrera 7B Bis # 132-11
8,fontibon,FTB,"4°40'41.67""N","74°8'37.75""W",2551,11,Fontibón,Urbana,De tráfico,Carrera 104 # 20 C - 31


(20, 10)

estacion            0
Sigla               0
Latitud             0
Longitud            0
Altitud (m)         0
Altura (m)          0
Localidad           0
Tipo de zona        0
Tipo de estación    0
Dirección           0
dtype: int64

estacion            object
Sigla               object
Latitud             object
Longitud            object
Altitud (m)          int64
Altura (m)           int64
Localidad           object
Tipo de zona        object
Tipo de estación    object
Dirección           object
dtype: object

# Preprocessing

## Preprocessing: fix datetime

In [15]:
air_data_df['DateTime'] = air_data_df['DateTime'].str.replace(
    ' 24:',
    ' 00:'
)

In [16]:
air_data_df['DateTime'] = pd.to_datetime(
    air_data_df['DateTime'],
    format='%d-%m-%Y %H:%M'
)

# Analysis

In [17]:
pollutants_list = ['PM2.5', 'PM10',  'NO', 'NO2', 'NOX', 'CO', 'OZONO']

In [18]:
FONT_SIZE_TICKS = 12
FONT_SIZE_TITLE = 20
FONT_SIZE_AXES = 16

## Analysis: histograms of different pollutants

In [19]:
def analyze_histograms(
    df: pd.DataFrame
) -> None:    
    def _interactive_histogram_plot(
        station: str,
        pollutant: str,
        bins: int
    ) -> None:
        data = df[df['Station']==station]
        x = data[pollutant].values 
        try:
            plt.figure(figsize=(12,6))
            plt.xlabel(f'{pollutant} concentration', fontsize=FONT_SIZE_AXES)
            plt.ylabel('Number of measurements', fontsize=FONT_SIZE_AXES)
            plt.hist(x, bins=bins)
            plt.title(f'Pollutant: {pollutant} - Station: {station}', fontsize=FONT_SIZE_TITLE)
            plt.xticks(fontsize=FONT_SIZE_TICKS)
            plt.yticks(fontsize=FONT_SIZE_TICKS)
            plt.show()
        except ValueError:
            print(f'Histogram cannot be shown for selected values: {e}')
    
    # Widget for picking the city
    station_selection = widgets.Dropdown(
        options=df.Station.unique(),
        description='Station'
    )

    # Widget for picking pollutant
    pollutant_selection = widgets.Dropdown(
        options=pollutants_list,
        description='Pollutant',
    )
    
    # Widget for picking number of bins in visualization
    bin_count_selection = widgets.IntSlider(
        min=1,
        max=1000,
        value=150,
        description='Bins count'
    )
    
    # Display the widget
    interact(
        _interactive_histogram_plot,
        station=station_selection,
        pollutant=pollutant_selection,
        bins=bin_count_selection
    )

In [20]:
analyze_histograms(
    df=air_data_df,
)

interactive(children=(Dropdown(description='Station', options=('USM', 'BOL', 'SUB', 'TUN', 'LFR', 'PTE', 'MAM'…

## Analysis: boxplots of pollutants

In [21]:
def analyze_boxplots(
    df: pd.DataFrame
) -> None:
    unique_stations = df['Station'].unique()
    
    def _interactive_boxplot_plot(
        pollutant: str,
        to_sort: bool
    ) -> None:
        if to_sort:
            medians = []
            for station in unique_stations:
                rows = df.loc[df['Station']==station, pollutant]
                if rows.notnull().sum() > 0:
                    median = rows.median()
                else:
                    median = -1000
                medians.append(median)
            orderInd = np.argsort(medians) 
        else:
            orderInd = range(len(unique_stations))
        
        try:
            plt.figure(figsize=(17,7))
            scale = 'linear'
            plt.yscale(scale)
            sns.boxplot(data=df, y=pollutant, x='Station', order=unique_stations[orderInd], color="seagreen")
            plt.title(f'Distributions of {pollutant}\n(sorted by median)', fontsize=FONT_SIZE_TITLE)
            plt.xlabel('Station', fontsize=FONT_SIZE_AXES)
            plt.ylabel(f'{pollutant} concentration', fontsize=FONT_SIZE_AXES)
            plt.xticks(fontsize=FONT_SIZE_TICKS)
            plt.yticks(fontsize=FONT_SIZE_TICKS)
            plt.show()
        except Exception as e:
            print(f'Boxplot cannot be shown for selected values: {e}')
        
    # Widget for picking a pollutant
    pollutant_selection = widgets.Dropdown(
        options=pollutants_list,
        description='Pollutant',
    )
    
    # Widget for sorting boxplots
    to_sort_selection = widgets.Dropdown(
        options=[True, False],
        value=True,
        description='Sort by Median'
    )
       
    # Display the widget
    interact(
        _interactive_boxplot_plot,
        pollutant=pollutant_selection,
        to_sort=to_sort_selection
    )

In [22]:
analyze_boxplots(air_data_df)

interactive(children=(Dropdown(description='Pollutant', options=('PM2.5', 'PM10', 'NO', 'NO2', 'NOX', 'CO', 'O…

## Analyze: scatterplot of pollutants

In [23]:
def analyze_scatterplots(
    df: pd.DataFrame
) -> None:
    df = df[pollutants_list]  # Take only the pollutants to scatter
    df_without_na = df.dropna()

    def _interactive_scatterplot(
        ox_feature: str,
        oy_feature: str
    ) -> None:
        x = df_without_na[ox_feature].values
        y = df_without_na[oy_feature].values        
        bins = [200, 200]

        hh, locx, locy = np.histogram2d(x, y, bins=bins)
        z = np.array([hh[np.argmax(a<=locx[1:]),np.argmax(b<=locy[1:])] for a,b in zip(x, y)])
        idx = z.argsort()
        x2, y2, z2 = x[idx], y[idx], z[idx]
        
        fig, ax = plt.subplots(figsize=(12, 6))
        s = ax.scatter(x2, y2, c=z2, cmap='jet', marker='.', s = 1)  
        
        ax.set_xlabel(f'{ox_feature} concentration', fontsize=FONT_SIZE_AXES)
        ax.set_ylabel(f'{oy_feature} concentration', fontsize=FONT_SIZE_AXES)

        ax.set_title(f'{ox_feature} vs. {oy_feature} (color indicates density of points)', fontsize=FONT_SIZE_TITLE)
        ax.tick_params(labelsize=FONT_SIZE_TICKS)

        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)

        fig.colorbar(
            s,
            cax=cax,
            cmap='jet',
            values=z2,
            orientation="vertical"
        )
        plt.show()
        
    # Widget for pollutant displayed on OX
    ox_feature_widget = widgets.Dropdown(
        options=pollutants_list,
        value='PM2.5',
        description='X-Axis'
    )
    
    # Widget for pollutant displayed on OY
    oy_feature_widget = widgets.Dropdown(
        options=pollutants_list,
        value='PM10',
        description='Y-Axis'
    )
    
    # Display the widget
    interact(
        _interactive_scatterplot,
        ox_feature=ox_feature_widget,
        oy_feature=oy_feature_widget
    )

In [24]:
analyze_scatterplots(
    air_data_df
)

interactive(children=(Dropdown(description='X-Axis', options=('PM2.5', 'PM10', 'NO', 'NO2', 'NOX', 'CO', 'OZON…

## Analyze: pairplot of pollutants

In [25]:
def analyze_pairplots(
    df: pd.DataFrame
) -> None:
    def _interactive_pairplot(
        pollutants: List[str]
    ) -> None:
        if not pollutants:
            pollutants = pollutants_list
        sns.pairplot(
            df[list(pollutants)],
            kind="hist",
            diag_kws={'alpha': 0.7, 'bins': 20}
        )
        plt.show()

    # Widget for selecting pollutants
    pollutants_widget = widgets.SelectMultiple(
        options=pollutants_list,
        value=['PM2.5', 'PM10'],
        description='Pollutants'
    )

    # Display the widget
    interact(
        _interactive_pairplot,
        pollutants=pollutants_widget
    )

In [26]:
analyze_pairplots(
    air_data_df
)

interactive(children=(SelectMultiple(description='Pollutants', index=(0, 1), options=('PM2.5', 'PM10', 'NO', '…

## Analyze: correlations between pollutants

In [27]:
def analyze_cross_corr_heatmaps(
    df: pd.DataFrame
) -> None:
    def _interactive_heatmap(
        pollutants: List[str],
        corr_method: str
    ) -> None:
        if not pollutants:
            pollutants = pollutants_list
        plt.figure(figsize=(10,10))
        sns.heatmap(
            df[list(pollutants)].corr(method=corr_method),
            square=True,
            annot=True,
            cbar=False,
            cmap="RdBu",
            vmin=-1,
            vmax=1
        )
        plt.title('Cross Correlation Matrix between Variables')
        plt.show()

    # Widget for selecting pollutants
    pollutants_widget = widgets.SelectMultiple(
        options=pollutants_list,
        value=['PM2.5', 'PM10'],
        description='Pollutants'
    )
    
    # Widget for selecting type of correlation to evaluate
    corr_method_widget = widgets.Dropdown(
        options=["pearson", "kendall", "spearman"],
        description='Correlation'
    )

    # Display the widget
    interact(
        _interactive_heatmap,
        pollutants=pollutants_widget,
        corr_method=corr_method_widget
    )

In [28]:
analyze_cross_corr_heatmaps(
    air_data_df
)

interactive(children=(SelectMultiple(description='Pollutants', index=(0, 1), options=('PM2.5', 'PM10', 'NO', '…

## Analyze: pollution over time

In [29]:
def analyze_time_series_plots(
    df: pd.DataFrame
) -> None:
    def _interactive_time_series_plot(
        station: str,
        pollutant: str,
        start_date: datetime.date,
        end_date: datetime.date
    ) -> None:
        data = df[
            (df['Station'] == station) &
            (df['DateTime'].dt.date >= start_date) &
            (df['DateTime'].dt.date <= end_date)
        ].sort_values(
            by='DateTime'
        )
        plt.figure(figsize=(12, 6))
        plt.plot(data["DateTime"], data[pollutant], '-')
        plt.title(f'Change of {pollutant}', fontsize=FONT_SIZE_TITLE)
        plt.ylabel(f'{pollutant} concentration', fontsize=FONT_SIZE_AXES)
        plt.xticks(rotation=20, fontsize=FONT_SIZE_TICKS)
        plt.yticks(fontsize=FONT_SIZE_TICKS)
        plt.show()
    
    # Widget for picking the station
    station_selection = widgets.Dropdown(
        options=df['Station'].unique(),
        description='Station'
    )

    # Widget for picking the pollutant
    pollutant_selection = widgets.Dropdown(
        options=pollutants_list,
        description='Pollutant',
    )

    # Widget for picking start/end dates for the timeframe to analyze
    start_date_widget = widgets.DatePicker(
        value=air_data_df['DateTime'].min().date(),
        description='Start date'
    )
    end_date_widget = widgets.DatePicker(
        value=air_data_df['DateTime'].max().date(),
        description='End date'
    )
    
    # Display the widget
    interact(
        _interactive_time_series_plot,
        station=station_selection,
        pollutant=pollutant_selection,
        start_date=start_date_widget,
        end_date=end_date_widget
    )

In [30]:
analyze_time_series_plots(
    air_data_df
)

interactive(children=(Dropdown(description='Station', options=('USM', 'BOL', 'SUB', 'TUN', 'LFR', 'PTE', 'MAM'…

## Analysis: draw maps

In [31]:
def parse_dms(coor: str) -> float:
    ''' Transforms strings of degrees, minutes and seconds to a decimal value
    
    Args:
        coor (str): String containing degrees in DMS format
        
    Returns:
        dec_coord (float): coordinates as a decimal value
    '''
    parts = re.split('[^\d\w]+', coor)
    degrees = float(parts[0])
    minutes = float(parts[1])
    seconds = float(parts[2]+'.'+parts[3])
    direction = parts[4]
    
    dec_coord = degrees + minutes / 60 + seconds / (3600)
    
    if direction == 'S' or direction == 'W':
        dec_coord *= -1
    
    return dec_coord


def add_extra_features(df: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    '''Adds new columns to the dataframe by joining it with another dataframe
    
    Args:
        df (pd.core.frame.DataFrame): The dataframe with the data.
    
    Returns:
        df (pd.core.frame.DataFrame): The updated dataframe with new columns.
    '''
    stations = pd.read_csv('data/csv/stations_loc.csv')
    stations = stations[['Sigla', 'Latitud', 'Longitud']]
    stations = stations.rename(columns={'Sigla': 'Station', 'Latitud': 'Latitude', 'Longitud': 'Longitude'})

    # This cell will convert the values in the columns 'Latitud' and 'Longitud' to 'float64' (decimal) datatype
    stations['Latitude'] = stations['Latitude'].apply(parse_dms)
    stations['Longitude'] = stations['Longitude'].apply(parse_dms)

    df = pd.merge(df, stations, on='Station', how='inner')

    # This cell will extract information from the 'datetime' column and generate months, day or week and hour columns
    df['day_of_week'] = pd.DatetimeIndex(df['DateTime']).day_name()
    df['hour_of_day'] = pd.DatetimeIndex(df['DateTime']).hour
    df.loc[df['hour_of_day']==0,'hour_of_day'] = 24
    return df

In [38]:
def color_producer(pollutant_type, pollutant_value):
    ''' This function returns colors based on the pollutant values to create a color representation of air pollution.    
    
    The color scale  for PM2.5 is taken from purpleair.com and it agrees with international guidelines
    The scale for other pollutants was created based on the limits for other pollutants to approximately
    correspond to the PM2.5 color scale. The values in the scale should not be taken for granted and
    are used just for the visualization purposes.
    
    Args:
        pollutant_type (str): Type of pollutant to get the color for
        pollutant_value (float): Value of polutant concentration
        
    Returns:
        pin_color (str): The color of the bucket
    '''
    all_colors_dict = {
        'PM2.5': {0: 'green', 12: 'yellow', 35: 'orange', 55.4: 'red', 150: 'black'},
        'PM10': {0: 'green', 20: 'yellow', 60: 'orange', 110: 'red', 250: 'black'},
        'CO': {0: 'green', 4: 'yellow', 10: 'orange', 20: 'red', 50: 'black'},
        'OZONE': {0: 'green', 60: 'yellow', 100: 'orange', 200: 'red', 300: 'black'},
        'NOX': {0: 'green', 40: 'yellow', 80: 'orange', 160: 'red', 300: 'black'},
        'NO': {0: 'green', 40: 'yellow', 80: 'orange', 160: 'red', 300: 'black'},
        'NO2': {0: 'green', 20: 'yellow', 40: 'orange', 80: 'red', 200: 'black'},
    }
    
    # Select the correct color scale, if it is not available, choose PM2.5
    colors_dict = all_colors_dict.get(pollutant_type, all_colors_dict['PM2.5'])
    thresholds = sorted(list(colors_dict.keys()))
    
    previous = 0
    for threshold in thresholds:
        if pollutant_value < threshold:
            bucket_size = threshold - previous
            bucket = (pollutant_value - previous) / bucket_size
            colors = list(Color(colors_dict[previous]).range_to(Color(colors_dict[threshold]), 11))
            pin_color = str(colors[int(np.round(bucket*10))])
            return pin_color
        previous = threshold

        
def create_map_with_plots(full_data: pd.core.frame.DataFrame, x_variable: str, y_variable: str='PM2.5') -> folium.Map:
    data = full_data[['Latitude', 'Longitude', y_variable, 'Station', x_variable]]
    data_grouped = data.groupby(['Station', x_variable]).agg(({y_variable: ['mean', 'std']}))
    ymin = data_grouped[y_variable]['mean'].min()
    ymax = data_grouped[y_variable]['mean'].max()

    grouped_means = defaultdict(dict)
    grouped_stds = defaultdict(dict)
    for index, row in data_grouped.iterrows():
        grouped_means[index[0]][index[1]] = row[0]
        grouped_stds[index[0]][index[1]] = row[1]

    for key in grouped_means:
        if (x_variable == 'day_of_week'):
            keys = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
            label = 'daily average'
        else:
            keys = list(grouped_means[key].keys())
            label = 'hourly average'

        values = []
        stds = []
        for subkey in keys:
            values.append(grouped_means[key][subkey])
            stds.append(grouped_stds[key][subkey])
        values = np.array(values)
        stds = np.array(stds)
        plt.plot(keys, values, '-o', label=label)
        plt.fill_between(keys, values - stds, values + stds, alpha=0.2)
        if y_variable == 'PM2.5':
            plt.plot(keys, [12]*len(keys),'--g', label='recommended level')
        plt.plot(keys, [np.average(values)]*len(keys), '--b', label='annual average')
        
        plt.ylim(ymin, ymax)
        plt.title(f'Station {key} avg. {y_variable} / {x_variable.split("_")[0]}')
        plt.ylabel(f'Avg. {y_variable} concentration')
        plt.xlabel(x_variable[0].upper() + x_variable[1:].replace('_', ' '))
        plt.legend(loc="upper left")
        if (x_variable == 'day_of_week'):
            plt.xticks(rotation=20)
        plt.savefig(f'results/img/{key}.png')

        plt.clf()
    
    data_grouped_grid = data.groupby('Station').agg(({y_variable: 'mean', 'Latitude': 'min', 'Longitude': 'min'}))
    
    data_grouped_grid_array = np.array(
        [
            data_grouped_grid['Latitude'].values,
            data_grouped_grid['Longitude'].values,
            data_grouped_grid[y_variable].values,
            data_grouped_grid.index.values
        ]
    ).T

    map3 = folium.Map(
        location=[data_grouped_grid_array[0][0], data_grouped_grid_array[0][1]],
        tiles='openstreetmap',
        zoom_start=11,
        width=1000,
        height=500
    )

    fg = folium.FeatureGroup(name="My Map")
    for lt, ln, pol, station in data_grouped_grid_array:
        fg.add_child(folium.CircleMarker(location=[lt, ln], radius = 15, popup=f"<img src='results/img/{station}.png'>",
        fill_color=color_producer(y_variable, pol), color = '', fill_opacity=0.5))
        map3.add_child(fg)
    return map3

In [39]:
# add some extra features like latitude and longitude to the data for mapping
enriched_raw_data = add_extra_features(air_data_df)

# choose a variable to calculate long-term averages for
x_variable = 'day_of_week' # Options ['day_of_week', 'hour_of_day']
# choose a pollutant which you are interested in
y_variable = 'PM10' # Options ['PM2.5', 'PM10', 'NO', 'NO2', 'NOX', 'CO', 'OZONE']

# generate a map representation of the data
create_map_with_plots(enriched_raw_data, x_variable, y_variable)

<Figure size 640x480 with 0 Axes>

In [34]:
print(datetime.datetime.now())

2023-07-06 22:32:59.952430
